<a href="https://colab.research.google.com/github/aonovoseltseva/computer_linguistics_25_26/blob/main/w2v_Novoseltseva_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [6]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 77.2 MB/s eta 0:00:00


## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [7]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [8]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [9]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


'glove-wiki-gigaword-300'

источники: 2024 Wikipedia (база данных на основе разных разделов википедии) + Gigaword 5 (база данных из разных новостных источников)
объем: 11.9B tokens, 1.2M vocab, 300d vectors, 1.6 GB download
использование: средняя размерность, поэтому обучение моделей для NLP целей, поиск похожих по смыслу списков слов

'glove-twitter-25'
источники: twitter
объем: 2B tweets, 27B tokens, 1.2M vocab, 25d vectors
использование: низкая размерность модели ограничивает применение, например, поиском похожих на запрос результатов по ключевым словам

'glove-twitter-50'
источники: twitter
объем: 2B tweets, 27B tokens, 1.2M vocab, 50d vectors
использование: поиск похожих товаров, первичная фильтрация*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [10]:
w2v_model = api.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [11]:
a = input('Введите любое слово ')
close_words = w2v_model.most_similar(a, topn = 5)
print('Близкие по смыслу слова:')
for word, score in close_words:
  print(f'{word}: {score}')

Введите любое слово овощи
Близкие по смыслу слова:
фрукты: 0.8991486430168152
котлеты: 0.7279103994369507
татуировки: 0.7276136875152588
орехи: 0.7274090647697449
напитки: 0.721818745136261


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [12]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [13]:
w2v_model_2 = Word2Vec(
    sentences = cooking_sentences,
    vector_size = 50,
    window = 3,
    min_count = 1,
    workers = 2,
    sg = 1,
)

In [14]:
print(f"Слова в словаре: {list(w2v_model_2.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [15]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = w2v_model_2.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [16]:
# Найдите слова, похожие на "духовка"
try:
  similar = w2v_model_2.wv.most_similar('духовка', topn = 5)
  print('Похожие на "духовка": ')
  for word, score in similar:
    print(f'{word}: {score}')
except KeyError:
  print('Слова "духовка" нет в словаре')
# Найдите слова, похожие на "овощи"
try:
  similar = w2v_model_2.wv.most_similar('овощи', topn = 5)
  print('Похожие на "овощи": ')
  for word, score in similar:
    print(f'{word}: {score}')
except KeyError:
  print('Слова "овощи" нет в словаре')

Похожие на "духовка": 
ингредиенты: 0.3198968470096588
десерт: 0.30644166469573975
холодильник: 0.27045494318008423
питание: 0.22426293790340424
пирог: 0.21422232687473297
Похожие на "овощи": 
мариновать: 0.2715907096862793
хлеб: 0.26912084221839905
гриль: 0.25464725494384766
фольга: 0.24094568192958832
сахар: 0.21084155142307281


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [17]:
ft_model = FastText(
    sentences= cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [18]:
try:
  similar = ft_model.wv.most_similar('варить', topn = 5)
  print('Похожие на "варить": ')
  for word, score in similar:
    print(f'{word}: {score}')
except KeyError:
  print('Слова "варить" нет в словаре')
try:
  similar = ft_model.wv.most_similar('духовка', topn = 5)
  print('Похожие на "духовка": ')
  for word, score in similar:
    print(f'{word}: {score}')
except KeyError:
  print('Слова "духовка" нет в словаре')
try:
  similar = ft_model.wv.most_similar('овощи', topn = 5)
  print('Похожие на "овощи": ')
  for word, score in similar:
    print(f'{word}: {score}')
except KeyError:
  print('Слова "овощи" нет в словаре')

Похожие на "варить": 
жарить: 0.5353310108184814
парить: 0.4804992079734802
месить: 0.3540874421596527
тушить: 0.34048697352409363
специи: 0.2621820569038391
Похожие на "духовка": 
взбивать: 0.45650428533554077
лимон: 0.3561227023601532
салат: 0.30499011278152466
курица: 0.3041205108165741
тост: 0.2943783700466156
Похожие на "овощи": 
жарить: 0.29603469371795654
фольга: 0.25739818811416626
морковь: 0.2296556979417801
соус: 0.217234805226326
торт: 0.20936326682567596


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [19]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = w2v_model_2.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('овщи')
compare_models('пираг')


Сравнение для слова: 'овщи'
  Word2Vec: слово не найдено
  FastText: ['овощи', 'питание']

Сравнение для слова: 'пираг'
  Word2Vec: слово не найдено
  FastText: ['тост', 'лимон']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [20]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [21]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [22]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [23]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")
    return similarity

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


np.float32(-0.08219737)

8. Сравните схожесть doc_2 и doc_4

In [24]:
compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


np.float32(-0.0362442)

9. Найдите самый похожий документ на doc_1

In [25]:
documents = range(5)
max_similarity = -1
most_similar_doc = None
for i in documents:
  if i == 1:
    continue
  similarity = compare_documents(1, i)
  print(f'doc_1 и doc_{i}: {similarity}')
  if similarity is not None and similarity > max_similarity:
    max_similarity = similarity
    most_similar_doc = i
print(f'Наиболее схожий документ с doc_1 — doc_{most_similar_doc}, similarity: {max_similarity}')

Схожесть doc_1 и doc_0: 0.2735
  doc_1: 1
  doc_0: 0
doc_1 и doc_0: 0.273516982793808
Схожесть doc_1 и doc_2: -0.0573
  doc_1: 1
  doc_2: 2
doc_1 и doc_2: -0.0573110431432724
Схожесть doc_1 и doc_3: 0.2031
  doc_1: 1
  doc_3: 3
doc_1 и doc_3: 0.20308849215507507
Схожесть doc_1 и doc_4: -0.2546
  doc_1: 1
  doc_4: 4
doc_1 и doc_4: -0.25457313656806946
Наиболее схожий документ с doc_1 — doc_0, similarity: 0.273516982793808


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [26]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

documents_tokenized = [word_tokenize(doc.lower()) for doc in documents]

model_10 = FastText(
    sentences= documents_tokenized,
    vector_size=10,
    window=3,
    min_count=1,
    workers=2
)
model_50 = FastText(
    sentences= documents_tokenized,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)
model_100 = FastText(
    sentences= documents_tokenized,
    vector_size=100,
    window=3,
    min_count=1,
    workers=2
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [30]:
def compare_models_ft(word_1, word_2, word_3):
    """Сравнивает представления слов в моделях FastText размерностей 10, 50, 100"""
# Сравнение для первого слова
    print(f"\nСравнение для слова: '{word_1}'")

#10
    try:
        model_10_similar = model_10.wv.most_similar(word_1, topn=2)
        print(f"  FastText10: {[w for w, _ in model_10_similar]}")
    except KeyError:
        print(f"  FastText10: слово не найдено")
#50
    try:
        model_50_similar = model_50.wv.most_similar(word_1, topn=2)
        print(f"  FastText50: {[w for w, _ in model_50_similar]}")
    except KeyError:
        print(f"  FastText50: слово не найдено")
#100
    try:
        model_100_similar = model_100.wv.most_similar(word_1, topn=2)
        print(f"  FastText100: {[w for w, _ in model_100_similar]}")
    except KeyError:
        print(f"  FastText100: слово не найдено")
        print(f"\nСравнение для слова: '{word_1}'")

# Сравнение для второго слова

    print(f"\nСравнение для слова: '{word_2}'")

#10
    try:
        model_10_similar = model_10.wv.most_similar(word_2, topn=2)
        print(f"  FastText10: {[w for w, _ in model_10_similar]}")
    except KeyError:
        print(f"  FastText10: слово не найдено")
#50
    try:
        model_50_similar  = model_50.wv.most_similar(word_2, topn=2)
        print(f"  FastText50: {[w for w, _ in model_50_similar]}")
    except KeyError:
        print(f"  FastText50: слово не найдено")
#100
    try:
        model_100_similar  = model_100.wv.most_similar(word_2, topn=2)
        print(f"  FastText100: {[w for w, _ in model_100_similar]}")
    except KeyError:
        print(f"  FastText100: слово не найдено")
        print(f"\nСравнение для слова: '{word_1}'")

# Сравнение для третьего слова

    print(f"\nСравнение для слова: '{word_3}'")

#10
    try:
        model_10_similar = model_10.wv.most_similar(word_3, topn=2)
        print(f"  FastText10: {[w for w, _ in model_10_similar]}")
    except KeyError:
        print(f"  FastText10: слово не найдено")
#50
    try:
        model_50_similar = model_50.wv.most_similar(word_3, topn=2)
        print(f"  FastText50: {[w for w, _ in model_50_similar]}")
    except KeyError:
        print(f"  FastText50: слово не найдено")
#100
    try:
        model_100_similar = model_100.wv.most_similar(word_3, topn=2)
        print(f"  FastText100: {[w for w, _ in model_100_similar]}")
    except KeyError:
        print(f"  FastText100: слово не найдено")
# Сравниваем для разных слов
compare_models_ft('learning', 'deep', 'networks')


Сравнение для слова: 'learning'
  FastText10: ['images', 'processes']
  FastText50: ['programming', 'amazing']
  FastText100: ['amazing', 'computer']

Сравнение для слова: 'deep'
  FastText10: ['data', 'amazing']
  FastText50: ['artificial', 'vision']
  FastText100: ['networks', 'interesting']

Сравнение для слова: 'networks'
  FastText10: ['python', 'images']
  FastText50: ['neural', 'is']
  FastText100: ['science', 'deep']
